# Importações

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from copy import copy, deepcopy
from typing import List

# Funções

In [ ]:
class Environment:
  def __init__(self, root, rel_path):
    self.root = root            # Pasta raiz
    self.rel_path = rel_path    # Caminho relativo

  def get_file_path(self):
    """
    Retorna caminho do arquivo.
    :return: Caminho do arquivo.
    """
    parts = [self.root]
    parts.extend(self.rel_path)
    return "/".join(parts)

  def clone(self, mode="shallow"):
    """
    Retorna uma cópia do objeto. O padrão é shallow copy.
    :param mode: Modo de cópia ("shallow" ou "deep").
    :return: Cópia do objeto.
    """
    if mode == "deep":
      return deepcopy(self)
    return copy(self)

def df_head(df: pd.DataFrame, num: int = 5) -> None:
  """
  Exibe as primeiras linhas do DataFrame.

  :param df: DataFrame a ser exibido.
  :param num: Número de linhas a serem exibidas.
  """
  display(df.head(num))
  print(f"Shape: {df.shape}")

def df_metrics2df(df: pd.DataFrame, columns: List[str], metrics: List[str] = None):
  """
    Calcula métricas estatísticas para colunas selecionadas de um DataFrame e retorna um novo DataFrame
    com as colunas originais como índices e as métricas como colunas.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo os dados numéricos.
        columns (List[str]): Lista de nomes de colunas numéricas do DataFrame `df` para calcular as métricas.
        metrics (List[str], optional): Lista de categorias de métricas a calcular.
            Pode conter qualquer combinação dos seguintes valores:
                - "centrality": Inclui média, mediana e moda.
                - "dispersion": Inclui amplitude, variância, desvio padrão e coeficiente de variação.
                - "position": Inclui os quartis Q1, Q2 (mediana) e Q3.
                - "skew": Inclui o coeficiente de assimetria (skewness).
                - "kurtosis": Inclui a curtose.
            Se None, todas as métricas são calculadas.

    Returns:
        pd.DataFrame: DataFrame com as colunas selecionadas como índice (linhas), e as métricas como colunas.

    Observações:
        - A moda pode retornar uma lista (caso haja múltiplos valores com a mesma frequência).
        - O coeficiente de variação é calculado como (desvio padrão / média), retornando NaN se a média for zero.
  """
  if metrics is None:
    metrics = ["centrality", "dispersion", "position", "skew", "kurtosis"]

  result = {}

  for col in columns:
    col_metrics = {}

    # Tendência Central
    if "centrality" in metrics:
      col_metrics["media"] = df[col].mean()
      col_metrics["mediana"] = df[col].median()
      modas = df[col].mode().tolist()
      col_metrics["moda"] = modas if len(modas) > 1 else modas[0] if modas else np.nan

    # Dispersão
    if "dispersion" in metrics:
      col_metrics["amplitude"] = df[col].max() - df[col].min()
      col_metrics["variancia"] = df[col].var()
      col_metrics["desv_pad"] = df[col].std()
      mean = df[col].mean()
      std_dev = df[col].std()
      col_metrics["coef_var"] = std_dev / mean if mean != 0 else pd.NA

    # Posição
    if "position" in metrics:
      col_metrics["q1"] = df[col].quantile(0.25)
      col_metrics["q2"] = df[col].quantile(0.50)
      col_metrics["q3"] = df[col].quantile(0.75)

    # Assimetria
    if "skew" in metrics:
      col_metrics["assimetria"] = df[col].skew()

    # Curtose
    if "kurtosis" in metrics:
      col_metrics["curtose"] = df[col].kurtosis()

    result[col] = col_metrics

  return pd.DataFrame(result).T # Transpõe o DataFrame antes de retornar


def df_classify_vars(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    '''
    Classifica colunas como qualitativas, quantitativas discretas ou contínuas.

    :param df: DataFrame com os dados.
    :param columns: Lista de colunas a classificar.
    :return: DataFrame com nome da coluna, tipo base e classificação estatística.
    '''
    resultado = []

    for col in columns:
        dtype = df[col].dtype
        classificacao = "Qualitativo"

        if pd.api.types.is_integer_dtype(dtype):
            classificacao = "Quantitativo Discreto"
        elif pd.api.types.is_float_dtype(dtype):
            classificacao = "Quantitativo Contínuo"

        resultado.append({
            "coluna": col,
            "dtype": str(dtype),
            "classificacao": classificacao
        })

    return pd.DataFrame(resultado)

def freq_table_qualitativas(df: pd.DataFrame, columns: List[str]) -> dict:
    """
    Gera e imprime tabelas de frequência (absoluta, relativa e acumuladas) para colunas qualitativas,
    com saída formatada.

    :param df: DataFrame com os dados.
    :param columns: Lista de colunas qualitativas.
    :return: Dicionário com DataFrames por coluna.
    """
    tabelas = {}

    for col in columns:
        serie = df[col]
        freq_abs = serie.value_counts().sort_index()
        freq_rel = (freq_abs / freq_abs.sum())
        freq_abs_ac = freq_abs.cumsum()
        freq_rel_ac = freq_rel.cumsum()

        tabela = pd.DataFrame({
            "Frequência Absoluta": freq_abs,
            "Frequência Relativa (%)": (freq_rel * 100),
            "Freq. Absoluta Acumulada": freq_abs_ac,
            "Freq. Relativa Acumulada (%)": (freq_rel_ac * 100)
        })

        tabelas[col] = tabela

        # Impressão formatada
        print(f"\nTabela de Frequência – Coluna: **{col}**")
        print(tabela.to_string())





# Setup do Ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Obtenção do Dataset

In [ ]:
# Ambiente de entrada
ROOT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/_all"
FILE = "all_merged.csv"
env_in = Environment(ROOT, [FILE])

df = pd.read_csv(env_in.get_file_path())

df_head(df)

# Classificação dos Atributos

Classifiquemos as variáveis do _dataset_ de acordo com seu tipo e com sua cardinalidade segundo as definições de Han _et al_ (2011):

As variáveis podem ser:



1.  `_ano`: quantitativo discreto
2.  `estado`: qualitativo nominal
3.  `_mes`: quantitativo discreto
4.  `car_c02_emitido`: quantitativo contínuo
5.  `cli_pressao_atm_med`: quantitativo contínuo
6.  `cli_temp_ar_med`: quantitativo contínuo
7.  `cli_temp_orvalho_med`: quantitativo contínuo
8.  `cli_umid_rel_med`: quantitativo contínuo
9.  `cli_umid_rel_min_max`: quantitativo contínuo
10. `cli_umid_rel_min_med`: quantitativo contínuo
11. `cli_umid_rel_min_min`: quantitativo contínuo
12. `cli_veloc_vento_max`: quantitativo contínuo
13. `cli_veloc_vento_med`: quantitativo contínuo
14. `que_area_queimada`: quantitativo contínuo
15. `que_focos_qtd`: quantitativo contínuo

In [ ]:
df = pd.read_csv("all_merged.csv")
df_head(df)

list_col = df.columns.tolist()

df_classify_vars(df, list_col)


#Cálculo de Medidas Estatísticas de Dados Quantitativos

In [ ]:
list_col = df.select_dtypes(include=["number"]).columns.tolist()
df_metrics2df(df, list_col)

#Tabela de Frequência

In [ ]:
list_col_qualitativo = ["_estado"]

freq_table_qualitativas(df, list_col_qualitativo)

# Análise gráfica

In [ ]:
# Convertendo colunas necessárias

df['_ano'] = df['_ano'].astype(int)
df['_mes'] = df['_mes'].astype(int)
df['que_area_queimada'] = pd.to_numeric(df['que_area_queimada'], errors='coerce')
df['que_focos_qtd'] = pd.to_numeric(df['que_focos_qtd'], errors='coerce')
df['car_c02_emitido'] = pd.to_numeric(df['car_c02_emitido'], errors='coerce')

#Gráfico 1 – Evolução Anual do CO₂ Emitido
df_ano = df.groupby('_ano')[['car_c02_emitido']].sum().reset_index()
sns.lineplot(data=df_ano, x='_ano', y='car_c02_emitido', marker='o')

#Gráfico 2 – Evolução Anual da Área Queimada
df_area = df.groupby('_ano')[['que_area_queimada']].sum().reset_index()
sns.lineplot(data=df_area, x='_ano', y='que_area_queimada', marker='o')

#Gráfico 3 – Correlação entre Área Queimada e CO₂ Emitido (por Estado)
df_estado = df.groupby('_estado')[['que_area_queimada', 'car_c02_emitido']].sum().reset_index()
matriz_corr_estado = df_estado[['que_area_queimada', 'car_c02_emitido']].corr()
sns.heatmap(matriz_corr_estado, annot=True, fmt=".2f")

# Gráfico 4 – Série Temporal Mensal (CO₂, Área Queimada, Focos)
df_mensal = df.groupby(['_ano', '_mes'])[['car_c02_emitido', 'que_area_queimada', 'que_focos_qtd']].sum().reset_index()
df_mensal['data'] = pd.to_datetime(df_mensal['_ano'].astype(str) + '-' + df_mensal['_mes'].astype(str))
sns.lineplot(data=df_mensal, x='data', y='car_c02_emitido', label='CO₂ Emitido')
sns.lineplot(data=df_mensal, x='data', y='que_area_queimada', label='Área Queimada')
sns.lineplot(data=df_mensal, x='data', y='que_focos_qtd', label='Focos de Queimada')


#Gráfico 5 – Correlação CO₂ Emitido vs Focos de Queimada
sns.scatterplot(data=df, x='que_focos_qtd', y='car_c02_emitido', hue='_estado', alpha=0.7)


#Gráfico 6 – Correlação Focos vs Área Queimada
sns.scatterplot(data=df, x='que_focos_qtd', y='que_area_queimada', hue='_estado', alpha=0.7)


#Gráfico 7 – Heatmap de Correlação Geral (Variáveis Numéricas)
colunas_correlacao = ['car_c02_emitido', 'que_area_queimada', 'que_focos_qtd']
matriz_corr = df[colunas_correlacao].corr()
sns.heatmap(matriz_corr, annot=True, fmt=".2f", cmap='coolwarm')


# Histograma da emissão de CO2
plt.figure(figsize=(10, 6))
sns.histplot(df['car_c02_emitido'], bins=30, kde=False, color='orange')
plt.title('Histograma das Emissões de CO₂')
plt.xlabel('CO₂ Emitido')
plt.ylabel('Frequência')
plt.tight_layout()
plt.show()

# Gráfico de densidade (KDE) da emissão de CO2
plt.figure(figsize=(10, 6))
sns.kdeplot(df['car_c02_emitido'], shade=True, color='red')
plt.title('Distribuição de Densidade das Emissões de CO₂')
plt.xlabel('CO₂ Emitido')
plt.ylabel('Densidade Estimada')
plt.tight_layout()
plt.show()



# Referências

HAN, Jiawei; KAMBER, Micheline; PEI, Jian. **Data mining: concepts and techniques**. 3. ed. Burlington: Morgan Kaufmann, 2011.